In [1]:
# 数据准备
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

plt.rcParams["font.family"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False
DATA_DIR = "./data/"
RESULT_DIR = "./results/"

# 加载数据
final_df = pd.read_csv(os.path.join(DATA_DIR, "final_cleaned_data.csv"))
model_df = final_df[final_df["Year"].between(2010, 2020)].copy()
# 人均GDP/能源消耗/年份 预测人均CO₂排放）
model_df = model_df[["Country", "Year", "Per_Capita_GDP", "Per_Capita_Energy", "Per_Capita_GHG"]].dropna()
print(f"样本量：{model_df.shape[0]}")

样本量：3465


In [2]:
# =回归模型：预测2021-2030人均CO₂排放
def predict_ghg():
    # 1. （以全球平均数据为例）
    global_avg = model_df.groupby("Year")[["Per_Capita_GHG"]].mean().reset_index()
    X = global_avg[["Year"]].values  # 年份
    y = global_avg["Per_Capita_GHG"].values  # 人均GHG（即CO₂排放）

    # 2. 构建多项式回归（比线性回归更贴合趋势）
    poly = PolynomialFeatures(degree=2)  # 2次多项式
    X_poly = poly.fit_transform(X)
    reg = LinearRegression()
    reg.fit(X_poly, y)

    # 3. 预测2021-2030年数据
    future_years = np.arange(2021, 2031).reshape(-1, 1)
    future_X_poly = poly.transform(future_years)
    future_ghg = reg.predict(future_X_poly)

    # 4. 模型评估（R²）
    y_pred = reg.predict(X_poly)
    r2 = r2_score(y, y_pred)
    print(f"回归模型R²：{r2:.3f}")

    # 5. 可视化结果
    plt.figure(figsize=(10, 6), dpi=150)
    # 历史
    plt.scatter(X, y, color="#2A9D8F", label="2010-2020历史数据", s=50)
    # 拟合曲线
    plt.plot(X, y_pred, color="#E76F51", linewidth=2.5, label=f"2次多项式拟合（R²={r2:.3f}）")
    # 预测数据
    plt.plot(future_years, future_ghg, color="#F4A261", linestyle="--", linewidth=2.5, marker="o", label="2021-2030预测")

    plt.title("2010-2030 全球人均CO₂排放趋势预测", fontsize=13)
    plt.xlabel("年份", fontsize=11)
    plt.ylabel("人均CO₂排放（吨/人）", fontsize=11)
    plt.xticks(np.concatenate([X.flatten(), future_years.flatten()], axis=0)[::2], fontsize=9)
    plt.grid(alpha=0.2)
    plt.legend(fontsize=10)
    plt.tight_layout()

    # 保存
    save_path = os.path.join(RESULT_DIR, "ghg_prediction.png")
    plt.savefig(save_path, dpi=150, facecolor="white")
    plt.close()
    print(f"路径：{save_path}")

    # 返回预测结果
    return pd.DataFrame({"Year": future_years.flatten(), "Predicted_Per_Capita_GHG": future_ghg})

# 执行预测
pred_result = predict_ghg()
print("\n===== 2021-2030人均CO₂排放预测结果 =====")
print(pred_result.round(4))

回归模型R²：0.835
路径：./results/ghg_prediction.png

===== 2021-2030人均CO₂排放预测结果 =====
   Year  Predicted_Per_Capita_GHG
0  2021                       0.0
1  2022                       0.0
2  2023                       0.0
3  2024                       0.0
4  2025                       0.0
5  2026                       0.0
6  2027                       0.0
7  2028                       0.0
8  2029                       0.0
9  2030                       0.0


C:\Users\过之远\AppData\Local\Temp\ipykernel_38888\3526094873.py:39: UserWarning: Glyph 8322 (\N{SUBSCRIPT TWO}) missing from font(s) SimHei.
  plt.tight_layout()
C:\Users\过之远\AppData\Local\Temp\ipykernel_38888\3526094873.py:39: UserWarning: Glyph 178 (\N{SUPERSCRIPT TWO}) missing from font(s) SimHei.
  plt.tight_layout()
C:\Users\过之远\AppData\Local\Temp\ipykernel_38888\3526094873.py:43: UserWarning: Glyph 8322 (\N{SUBSCRIPT TWO}) missing from font(s) SimHei.
  plt.savefig(save_path, dpi=150, facecolor="white")
C:\Users\过之远\AppData\Local\Temp\ipykernel_38888\3526094873.py:43: UserWarning: Glyph 178 (\N{SUPERSCRIPT TWO}) missing from font(s) SimHei.
  plt.savefig(save_path, dpi=150, facecolor="white")


In [ ]:
# 聚类模型：K-Means分组国家
def cluster_countries():
    # 1. 数据预处理（取2020年数据，按国家聚合）
    country_data = model_df[model_df["Year"] == 2020].groupby("Country")[
        ["Per_Capita_Energy", "Per_Capita_GHG"]
    ].mean().reset_index()
    # 标准化特征（K-Means对量纲敏感）
    scaler = StandardScaler()
    X_cluster = scaler.fit_transform(country_data[["Per_Capita_Energy", "Per_Capita_GHG"]])

    # 2. 构建K-Means模型（聚为3类）
    kmeans = KMeans(n_clusters=3, random_state=42)
    country_data["Cluster"] = kmeans.fit_predict(X_cluster)

    # 3. 可视化聚类结果
    plt.figure(figsize=(10, 6), dpi=150)
    colors = ["#2A9D8F", "#E76F51", "#F4A261"]
    for cluster in range(3):
        cluster_data = country_data[country_data["Cluster"] == cluster]
        plt.scatter(
            cluster_data["Per_Capita_Energy"],
            cluster_data["Per_Capita_GHG"],
            color=colors[cluster],
            label=f"分组{cluster+1}",
            s=80,
            alpha=0.8
        )
        # 标注国家名
        for _, row in cluster_data.iterrows():
            plt.text(
                row["Per_Capita_Energy"] + 0.001,
                row["Per_Capita_GHG"] + 0.0001,
                row["Country"],
                fontsize=8
            )

    plt.title("2020年国家能源消耗-排放强度聚类结果", fontsize=13)
    plt.xlabel("人均能源消耗（万吨油当量/万人）", fontsize=11)
    plt.ylabel("人均CO₂排放（百万吨/万人）", fontsize=11)
    plt.grid(alpha=0.2)
    plt.legend(fontsize=10)
    plt.tight_layout()

    # 保存
    save_path = os.path.join(RESULT_DIR, "country_clustering.png")
    plt.savefig(save_path, dpi=150, facecolor="white")
    plt.close()
    print(f"路径：{save_path}")

    # 返回聚类结果
    return country_data[["Country", "Cluster"]]

# 执行聚类
cluster_result = cluster_countries()

print(cluster_result)

C:\Users\过之远\AppData\Local\Temp\ipykernel_38888\2391749153.py:42: UserWarning: Glyph 8322 (\N{SUBSCRIPT TWO}) missing from font(s) SimHei.
  plt.tight_layout()
C:\Users\过之远\AppData\Local\Temp\ipykernel_38888\2391749153.py:42: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
C:\Users\过之远\AppData\Local\Temp\ipykernel_38888\2391749153.py:46: UserWarning: Glyph 8322 (\N{SUBSCRIPT TWO}) missing from font(s) SimHei.
  plt.savefig(save_path, dpi=150, facecolor="white")


✅ 聚类图保存成功 | 路径：./results/country_clustering.png

===== 国家聚类结果 =====
                         Country  Cluster
0                    Afghanistan        0
1                         Africa        0
2    Africa Eastern and Southern        0
3     Africa Western and Central        0
4                        Albania        0
..                           ...      ...
310                        World        0
311                        Yemen        0
312                  Yemen, Rep.        0
313                       Zambia        0
314                     Zimbabwe        0

[315 rows x 2 columns]
